# NDVI EVI Statistics Processing Script

**Authors:** Gabriel Ortega & Michela Perrone

## Libraries

In [1]:
import ee
import time
import os
ee.Authenticate()
ee.Initialize(project='gortega-research')

from datetime import datetime

## 1. Settings

In [2]:
# Polygons collection
table = ee.FeatureCollection("projects/gortega-research/assets/wdpa_workdata")  # Explicitly define WDPA polygons
# Unique polygons identifier
uniqueID = 'wdpaid'

# Rasters collection
collection = 'MODIS/061/MOD13Q1'

# Set years of interest
startYear = 2020
endYear = 2024

# Select bands of interest
bands = ['NDVI', 'EVI']
qualityBand = 'SummaryQA'
qualityValue = 0
scaleFactor = 0.0001

# Full bounding box of the study area
xmin, ymin, xmax, ymax = -180, 90, 180, -90

# Range
rangestart = 572
rangeend = 1025

# Get the current date in YYYY-MM-DD format
current_date = datetime.now().strftime('%Y-%m-%d')
export_folder = f'NDVI_EVI_wdpa_{current_date}'

## 2. Function(s)

In [3]:
def processData(year, features, imageCollection):

    # Check if the FeatureCollection is empty
    if features.size().getInfo() == 0:
        return ee.FeatureCollection([])  # Return an empty FeatureCollection

    # 2.1. Image Processing:
    processedImage = imageCollection.filterBounds(features) \
        .reduce(ee.Reducer.percentile([90])) \
        .multiply(scaleFactor)

    # Check if the processed image is empty
    if processedImage.bandNames().size().getInfo() == 0:
        return ee.FeatureCollection([]) # Return empty if no images were found

    # 2.2. Feature Generation:
    result = features.map(lambda feature: ee.Feature(
        None,
        processedImage.reduceRegion(
            reducer=ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True),
            geometry=feature.geometry(),
            scale=250,
            maxPixels=1e13,
            bestEffort=True,
            tileScale=16
        ).set(uniqueID, feature.get(uniqueID)).set('year', year)
    ))
    return result

## 3. Execution

In [4]:
# Loop over whichBox
for whichBox in range(rangestart, rangeend): #[459, 460, 490, 491, 492, 522]: #
    # Initialize bbox variable
    if whichBox == 0:
        # Full bounding box
        bbox = ee.Geometry.BBox(xmin, ymin, xmax, ymax)
    else:
        # Grid configuration
        xSegments, ySegments = 32, 32

        # Calculate step sizes
        xStep = (xmax - xmin) / xSegments  # 45° per column
        yStep = (ymax - ymin) / ySegments  # -22.5° per row

        # Convert whichBox to grid indices
        adjustedIndex = whichBox - 1
        row = adjustedIndex // xSegments  # 0-3 (bottom to top)
        col = adjustedIndex % xSegments  # 0-3 (left to right)

        # Reverse row order (original grid is top to bottom)
        yIndex = (ySegments - 1) - row

        # Calculate coordinates
        xStart = xmin + col * xStep
        xEnd = xStart + xStep
        yStart = ymin + yIndex * yStep
        yEnd = yStart + yStep

        bbox = ee.Geometry.BBox(xStart, yStart, xEnd, yEnd)

    # Filter polygons dataset as required
    filteredPols = table.filterBounds(bbox)

    # Check if filteredPols is empty before processing
    if filteredPols.size().getInfo() > 0:

        # Apply simplify to the geometry of each feature using map
        simplifiedPols = filteredPols.map(lambda feature: feature.simplify(maxError=50))

        # Loop through years and process/export data for each year
        for year in range(startYear, endYear + 1):


            # Pre-filter the image collection for the specific year
            prefilteredCollection = ee.ImageCollection(collection) \
            .select(bands + [qualityBand]) \
            .filterBounds(simplifiedPols) \
            .filterDate(f'{year}-01-01', f'{year+1}-01-01') \
            .map(lambda image: image.updateMask(image.select(qualityBand).eq(qualityValue)).select(bands))

            features_this_year = processData(year, simplifiedPols, prefilteredCollection)

            # Name of output file including the year
            fileName = f'mean_sd_NDVI_EVI_WDPA_{xStart}_{yStart}_{xEnd}_{yEnd}_{year}_bbox{whichBox}'

            ############################################################################################################################
            # 4. EXPORT RESULTS

            if features_this_year.size().getInfo() > 0:
                task = ee.batch.Export.table.toDrive(
                    collection=features_this_year,
                    description=fileName,
                    fileFormat='CSV',
                    folder=export_folder
                )
                task.start()
                print(f"Exporting: {fileName}")  # Add a print statement
                time.sleep(10)  # Sleep between task submissions
            else:
                print(f"No data to export for {fileName}") # Add message for empty exports

    else:
        print(f"No polygons in bbox{whichBox} to process.") # Add message for empty bounding box

Exporting: mean_sd_NDVI_EVI_WDPA_123.75_11.25_135.0_5.625_2020_bbox572
Exporting: mean_sd_NDVI_EVI_WDPA_123.75_11.25_135.0_5.625_2021_bbox572
Exporting: mean_sd_NDVI_EVI_WDPA_123.75_11.25_135.0_5.625_2022_bbox572
Exporting: mean_sd_NDVI_EVI_WDPA_123.75_11.25_135.0_5.625_2023_bbox572
Exporting: mean_sd_NDVI_EVI_WDPA_123.75_11.25_135.0_5.625_2024_bbox572
No polygons in bbox573 to process.
No polygons in bbox574 to process.
No polygons in bbox575 to process.
No polygons in bbox576 to process.
No polygons in bbox577 to process.
No polygons in bbox578 to process.
No polygons in bbox579 to process.
No polygons in bbox580 to process.
No polygons in bbox581 to process.
No polygons in bbox582 to process.
No polygons in bbox583 to process.
Exporting: mean_sd_NDVI_EVI_WDPA_-101.25_16.875_-90.0_11.25_2020_bbox584
Exporting: mean_sd_NDVI_EVI_WDPA_-101.25_16.875_-90.0_11.25_2021_bbox584
Exporting: mean_sd_NDVI_EVI_WDPA_-101.25_16.875_-90.0_11.25_2022_bbox584
Exporting: mean_sd_NDVI_EVI_WDPA_-101.25_

Before running the Earth Engine code, you need to authenticate with your Google account to grant Colab access to Earth Engine. Run the next cell and follow the instructions.